# Dependencies

In [ ]:
!pip install -qU langchain-groq
!pip install -U ddgs
!pip install -qU duckduckgo-search langchain-community
!pip install -qU langchain-community wikipedia
!pip install -qU youtube_search
!pip install -qU  youtube-transcript-api
!pip install pytube

# LLM

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="qwen/qwen3-32b",
    api_key="my_api_key"
)

# Tools

In [158]:
tools = []

Searching on web using duck duck go search.


In [159]:
from langchain_community.tools import DuckDuckGoSearchRun

tools.append(DuckDuckGoSearchRun())

Retreving content from wikipedia using WikipediaQueryRun

In [160]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikiSearch = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

tools.append(wikiSearch)

YouTube search tool using YoutubeLoader and YouTubeSearchTool

In [165]:
import ast # convert string into actual python list
from langchain_core.tools import tool
from langchain_community.tools import YouTubeSearchTool
from langchain_community.document_loaders import YoutubeLoader

@tool("youtube_search", description="finds youtube videos using YoutubeSearchTool and then fetches content of the link using YoutubeLoader")
def fetch_youtube_content(query: str) -> str:
    """
    Fetches the full transcript text from a YouTube video link.
    """
    youtube_tool = YouTubeSearchTool()

    raw_result = youtube_tool.run(query)
    video_ids = ast.literal_eval(raw_result)
    links = [vid for vid in video_ids]

    all_text = ""
    i = 0
    for link in links:
      all_text += f"Video {i + 1} Content: "
      loader = YoutubeLoader.from_youtube_url(
          link,
          add_video_info=False,
      )
      doc = loader.load()
      all_text += doc[0].page_content + "\n"
      i = i + 1

    return all_text


tools.append(fetch_youtube_content)

Google search api key

In [ ]:
from langchain_community.utilities import SearchApiAPIWrapper
from langchain_core.tools import Tool

SEARCHAPI_API_KEY = "my_api_key"

# 1) Google Web Search
google_web = SearchApiAPIWrapper(
    searchapi_api_key=SEARCHAPI_API_KEY,
    engine="google",            # standard Google web search
)

google_web_tool = Tool(
    name="google_web_search",
    description="General Google web search via SearchApi.io.",
    func=google_web.run,
)

# 2) Google News
google_news = SearchApiAPIWrapper(
    searchapi_api_key=SEARCHAPI_API_KEY,
    engine="google_news",       # Google News vertical
)

google_news_tool = Tool(
    name="google_news_search",
    description="Latest news headlines and articles via Google News (SearchApi.io).",
    func=google_news.run,
)

# 3) Google Scholar
google_scholar = SearchApiAPIWrapper(
    searchapi_api_key=SEARCHAPI_API_KEY,
    engine="google_scholar",    # Google Scholar vertical
)

google_scholar_tool = Tool(
    name="google_scholar_search",
    description="Academic papers and scholarly articles via Google Scholar (SearchApi.io).",
    func=google_scholar.run,
)

# Adding to my tools list
tools.append(google_web_tool)
tools.append(google_news_tool)
tools.append(google_scholar_tool)

# Prompt using lang chain hub

In [163]:
from langchain_classic import hub

prompt = hub.pull("hwchase17/react")

# Agent = LLM + Tool + Prompt

In [170]:
from langchain_classic.agents import initialize_agent

agent = initialize_agent(
    tools=tools,
    llm=llm,
    prompt=prompt,
    verbose=True
)

print(agent.run("Best videos to understand Langchain."))



> Entering new AgentExecutor chain...
<think>
Okay, I need to find the best videos to understand LangChain. Let me start by recalling what LangChain is. It's a framework for developing applications with language models, right? So, maybe there are official tutorials or popular YouTube videos that explain it well.

First, I should use the youtube_search tool to look for videos about LangChain. The user wants the best videos, so maybe the most recent or highly viewed ones. Let me try searching for "LangChain tutorial" or "LangChain explained".

Wait, the user mentioned "best videos", so maybe I should look for comprehensive guides or official documentation videos. Also, checking the number of views and likes could help determine popularity. Let me perform a YouTube search with the query "Best LangChain tutorials" to see what comes up.

After getting the search results, I should look at the video titles and descriptions to pick the most relevant ones. If there's an official LangChain cha